<a href="https://colab.research.google.com/github/adityasingh345/Thermophysical-Property-Melting-point-/blob/main/melting_point_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# IMPORTANT: SOME KAGGLE DATA SOURCES ARE PRIVATE
# RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES.
import kagglehub
kagglehub.login()


In [ ]:
# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

melting_point_path = kagglehub.competition_download('melting-point')
aliffaagnur_melting_point_chemical_dataset_path = kagglehub.dataset_download('aliffaagnur/melting-point-chemical-dataset')
aliffaagnur_optuna_study_3_models_other_optuna_study_3_models_3_path = kagglehub.model_download('aliffaagnur/optuna-study-3-models/Other/optuna-study-3-models/3')

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
# for dirname, _, filenames in os.walk('/kaggle/input'):
#     for filename in filenames:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

RDkit is an open source cheminformatics library - basically a toolkit that helps scientists and devloper work with chemical structures using code

In [ ]:
!pip install rdkit

In [ ]:
import shutil
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

import optuna
from sklearn.feature_selection import SelectFromModel
from sklearn.preprocessing import PowerTransformer # data transformation technique that makes your data more gaussian forma or normal form
# which often helps to imporve the accuracy
from sklearn.model_selection import KFold, cross_val_score, StratifiedKFold
from sklearn.metrics import mean_absolute_error, make_scorer, mean_squared_error
from rdkit import Chem
from rdkit.Chem import Descriptors, Crippen, rdMolDescriptors, MACCSkeys, RDKFingerprint, rdFingerprintGenerator
from rdkit.Chem.AtomPairs import Pairs, Torsions

from rdkit import RDLogger
RDLogger.DisableLog('rdApp.*')

try:
    from rdkit.Avalon import pyAvalonTools
    avalon_available = True
except ImportError:
    avalon_available = False

import plotly.io as pio
pio.renderers.default = "iframe_connected"

from IPython.display import clear_output

import xgboost

chem give you access to molecular representation mol=Chem.molFromSmiles("cco") ethanol. descriptors functions to calculate the physiochemical properties of molecules. crippen-> computes crippen descriptors , mailyLogP(octanol-water partition coefficient) and molecular Refractivity(MR) . MACCSKeys-> generates maccskeys fingerprints : a classic type of binary molecular fingerprint (166but vector) each bit represnts the presence /absemce of a predefined substructers . rdmodeldescriptors-> a collection of high level of molecular descriptions including morgan Fingerprints, topological torsion fingerprints, atom pair fingerprints.rdmolops/rdkfingerprint-> rdkfingerprint i sthe original rdkit fingerprint algorithm

In [ ]:
# loading the data
train = pd.read_csv('/kaggle/input/melting-point/train.csv')
test = pd.read_csv('/kaggle/input/melting-point/test.csv')

bradley = pd.read_excel('/kaggle/input/melting-point-chemical-dataset/BradleyMeltingPointDataset.xlsx')
bradleyplus = pd.read_excel('/kaggle/input/melting-point-chemical-dataset/BradleyDoublePlusGoodMeltingPointDataset.xlsx')

# hum log jo hamara competition ka data hai usme se compound and unka melting point rakh lenge and us particular compound
# se related features hum rdkit ke through extract karenge
train = train[['SMILES', 'Tm']]
test = test[['id', 'SMILES']]

train.shape, test.shape, bradley.shape, bradleyplus.shape

In [ ]:
bradley.head()

In [ ]:
# tm clesius me diya ha we will convert it into kelvin
# to convert it into kelvin
bradley['Tm'] = bradley['mpC'].map(lambda x : x + 273.15)
# same with bradleyplus
bradleyplus['Tm'] = bradleyplus['mpC'].map(lambda x : x + 273.15)

# hum sirf smiles and unka tm rakh rahe hai
bradley = bradley[['smiles', 'Tm']]
bradleyplus = bradleyplus[['smiles', 'Tm']]

bradley_df = pd.concat((bradley, bradleyplus), axis = 0)
bradley_df = bradley_df.rename(columns = {'smiles': 'SMILES'})

bradley_df

In [ ]:
# we will merge the bradley data and train data together

merge = pd.concat((train ,bradley_df), axis = 0)

merge

In [ ]:
# there are chances that the final dataset merge can have duplicate valeus so remove thpse
merge  = merge.drop_duplicates(subset = ['SMILES', 'Tm']).reset_index(drop = True)

merge

display(f"{merge.duplicated(subset = ['SMILES', 'Tm']).sum()}")

pehle hume smajhna hoga ki secriptor_list dikhta kaisa hai , list where each item is a pair (a tuple) containing; -> a descriptor name , -> the function that calls it . (for name , func in descriptor_list) -> iterates through this list har step pe it unpacks one of the pairs, in first loop name will be molwt and function will be the molwt function , in second loop name will be 'numhdonors' and func will be the numhdonors and so on.
name: this becomes the key in the new dictionary funct(mol) it calls the function that was just unpacked and passes your mol to it, the result becomes the value {..} the curly braces tell python to collect all these generated keys-value pairs and put them together into a single new dictionary .

In [ ]:
def extract_all_descriptors(df, SMILES):
    descriptor_list = Descriptors.descList
    # Descriptors.descList -> contain all descriptors names and function
    descriptors = [desc[0] for desc in descriptor_list]


    result = []
    for smi in df[SMILES]:
        mol = Chem.MolFromSmiles(smi)

        if mol is None:
            row = {name: None for name , func in descriptor_list}
        else:
            row = {name: func(mol) for name , func in descriptor_list}
            result.append(row)

    df_descriptor = pd.DataFrame(result) # converting into a dataset
    df_result = pd.concat((df, df_descriptor), axis = 1)

    return df_result

merge = extract_all_descriptors(merge, 'SMILES')

test = extract_all_descriptors(test, 'SMILES')

merge = merge.dropna().reset_index(drop = True)
test = test.dropna().reset_index(drop = True)

merge


In [ ]:
def extract_all_fingerprint(df, SMILES, morgan_radius = 2, morgan_nbits = 1024):

    fps_data = []  # --> STORE NEW FEATURES DATA

    # DEFINE MORGAN GENERATOR
    morgan_gen = rdFingerprintGenerator.GetMorganGenerator(radius = morgan_radius, fpSize = morgan_nbits, countSimulation = True, includeChirality = False)

    fcfp = rdFingerprintGenerator.GetMorganFeatureAtomInvGen()
    fcfp_gen = rdFingerprintGenerator.GetMorganGenerator(radius = morgan_nbits, fpSize = morgan_nbits, atomInvariantsGenerator = fcfp, countSimulation= True, includeChirality = False)

    atom_gen = rdFingerprintGenerator.GetAtomPairGenerator(fpSize = 2048, countSimulation= True, includeChirality = False)

    # ITERATE EVERY SAMPLE OF SMILES FEATURES
    for smiles in df[SMILES]:
        mol = Chem.MolFromSmiles(smiles)

        if mol is None:
            print(smiles, 'is Invalid!')
            fps_data.append({})
            continue

        # STORE NEW FEATURE FOR EACH SAMPLES CREATED
        feature_rows = {}

        # MORGAN FINGERPRINT (ECFP)
        morgan_fp = morgan_gen.GetFingerprint(mol)
        for i in range(morgan_nbits):
            feature_rows[f"Morgan_{i}"] = morgan_fp[i]

        # FUNCTIONAL-CLASS FINGERPRINT (FCFP)
        fc_fp = fcfp_gen.GetFingerprint(mol)
        for i in range(morgan_nbits):
            feature_rows[f"FCFP_{i}"] = fc_fp[i]

        # MACCS KEYS (166 BITS)
        maccs_fp = MACCSkeys.GenMACCSKeys(mol)
        for i in range(len(maccs_fp)):
            feature_rows[f"MACCS_{i}"] = int(maccs_fp[i])



        # RDKIT FINGERPRINT
        rdkit_fp = RDKFingerprint(mol)
        for i in range(len(rdkit_fp)):
            feature_rows[f"RDKIT_{i}"] = int(rdkit_fp[i])




        fps_data.append(feature_rows)

    print(f'There are {morgan_nbits} Morgan Fingerprint Features')
    print(f'There are {len(maccs_fp)} MACCS Keys Features')
    print(f'There are {len(rdkit_fp)} RDKIT Fingerprint Features')

    # MERGE REAL DATA WITH EXTRACTED FEATURES
    fps_df = pd.DataFrame(fps_data)
    df_result = pd.concat((df, fps_df), axis = 1)

    return df_result


# APPLY FUNCTION
merge = extract_all_fingerprint(merge, 'SMILES')
test  = extract_all_fingerprint(test, 'SMILES')

merge

**A molecular fingerprint is a way to represent a complex chemical structure as a simple list of numbers (a vector), making it easy for a computer to search, compare, and build machine learning models.** Using a generator is the modern and efficient way to do this in RDKit. Instead of re-typing the settings for every molecule, you create one generator object that holds your "recipe" for making fingerprints. This ensures consistency and is faster when you're processing a large dataset.
****There are 1024 Morgan Fingerprint Features
There are 167 MACCS Keys Features
There are 2048 RDKIT Fingerprint Features****

In [ ]:
# splitting the training and testing data
x  = merge.drop(labels = ['SMILES', 'Tm'], axis = 1)

y = merge['Tm']

x_test = test.drop(labels = ['SMILES', 'id'], axis = 1)

x.shape, y.shape, x_test.shape, type(x)

-> verbosity: 0 - controls the amount of logging XGBoost does
-> objective - defines the type of problem(what the learns to minimize).
'reg:squarederror'- minimizes mean squared error, 'reg:pseudohubererror- a robust loss'
->tree-method - gpu_hist -> uses gpu for histogram-based tree building
->predictor- ensures predictions run on GPU
-> devices: cuda
-> booster: 'gbtree' - defines the type of base model XGbost uses


In [ ]:
def objective(trial):
    xgb_params = {
        "verbosity": 0,
        "objective": trial.suggest_categorical("objective", ['reg:squarederror', 'reg:pseudohubererror']),
        "tree_method": "gpu_hist",
        'predictor' : 'gpu_predictor',
        'device' : 'cuda',
        "eval_metric": "rmse",
        "booster": "gbtree",
        'n_estimators' : 10_000,
        "max_depth": trial.suggest_int("max_depth", 3, 7),
        "learning_rate": trial.suggest_float("learning_rate", 3e-3, 0.3, log=True),
        "min_child_weight": trial.suggest_int("min_child_weight", 1, 10),
        "subsample": trial.suggest_float("subsample", 0.5, 1.0),
        "colsample_bytree": trial.suggest_float("colsample_bytree", 0.5, 1.0),
        "gamma": trial.suggest_float("gamma", 0, 5),
        "lambda": trial.suggest_float("lambda", 0.1, 20.0, log=True),
        "alpha": trial.suggest_float("alpha", 0.1, 20.0, log=True),
    }


    kfold = KFold(n_splits= 5 , shuffle =True, random_state = 2000)
    rmse_scores = []

    for train_idx, valid_idx in kf.split(x):
        X_train , X_valid = x.iloc[train_idx], x.iloc[valid_idx]
        y_train, y_valid = y[train_idx], y[valid_idx]


        dtrain = xgboost.DMatrix(X_train, label = y_train)
        dvalid = xgboost.DMatrix(X_valid, label = y_valid)

        model = xgboost.train(
            xgb_params,
            dtrain,
            num_boost_round = 10000,
            evals= [(dvalid, "validation")],
            early_stopping_rounds = 100,
            verbose_eval = False,
        )
        pred = model.predict(dvalid)
        rmse = mean_squared_error(y_valid, pred, squared =False)
        rmse_scores.appen(rmse)

    return np.mean(rmse_scores)

In [ ]:
# start optuna to finding the best parameters for our models
shutil.copy(src = '/kaggle/input/optuna-study-3-models/other/optuna-study-3-models/3/xgb_study.db', dst = '/kaggle/working/xgb_study.db')
study = optuna.create_study(direction = "minimize", study_name = 'xgb_study', storage="sqlite://////kaggle/working/xgb_study.db", load_if_exists = True)

clear_output(wait=True)

print(f'Training Complete! Congrats!')
print(f'Total Number of Trials : {len(study.trials)}\n')

print("Best Trial", study.best_trial.number)
print("Best MAE:", study.best_value)
print("Best Params:", study.best_trial.params)

In [ ]:
best_params = {
    'max_depth' : 6,
    'eta' : 0.1,
    'tree_method' : 'hist',
    'eval_metric' : 'mae'
}

best_params

In [ ]:
'''{'objective': 'reg:squarederror',
 'max_depth': 3,
 'learning_rate': 0.01688021212211354,
 'min_child_weight': 5,
 'subsample': 0.8943127227676447,
 'colsample_bytree': 0.590582609011384,
 'gamma': 0.01132850232872052,
 'lambda': 4.445806747037075,
 'alpha': 0.1292033669407927,
 'eval_metric': 'mae',
 'device': 'cpu'}'''

In [ ]:
best_params.update({
    'n_estimators': 10_000,
    'tree_method': 'gpu_hist',
    'predictor': 'gpu_predictor',
    'device': 'cuda',
    'eval_metric': 'mae'  # since your best metric was MAE
})

In [ ]:
X_train = merge.drop(columns=['SMILES', 'Tm'])
y_train = merge['Tm']
X_test = test.drop(columns=['SMILES'], errors='ignore')

In [ ]:
import xgboost as xgb
best_params.pop("predictor",None)
final_model = xgb.XGBRegressor(**best_params)

final_model.fit(
    X_train, y_train,
    eval_set=[(X_train, y_train)],
    verbose=False
)

In [ ]:
X_train.shape

In [ ]:
# X_test = X_test.drop(columns = ['id'])

In [ ]:
print(X_test.shape, X_train.shape)

In [ ]:
print("Extra columns in X_test:", set(X_test.columns) - set(X_train.columns))
print("Missing columns in X_test:", set(X_train.columns) - set(X_test.columns))

In [ ]:
X_test = X_test.drop(columns=["id"])

In [ ]:
print(X_test.shape)

In [ ]:
y_pred = final_model.predict(X_test.to_numpy())

In [ ]:
y_pred2 = final_model.predict(X_train)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

print("MSE:", mean_squared_error(y_train, y_pred2))
print("MAE:", mean_absolute_error(y_train, y_pred2))
print("R2:", r2_score(y_train, y_pred2))

In [ ]:
from sklearn.model_selection import cross_val_score

# Use negative MSE because cross_val_score always maximizes the score
cv_scores = cross_val_score(final_model, X_train, y_train, cv=5, scoring='neg_mean_squared_error')
cv_rmse = np.sqrt(-cv_scores)
print("Cross-validated RMSE:", cv_rmse.mean())

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, median_absolute_error, max_error, explained_variance_score

median_err = median_absolute_error(y_train, y_pred2)
max_err = max_error(y_train, y_pred2)
expl_var = explained_variance_score(y_train, y_pred2)

print(median_err)
print(max_err)
print(expl_var)

In [ ]:
sample = pd.read_csv('/kaggle/input/melting-point/sample_submission.csv')
sample.head()

In [ ]:
assert len(sample) == len(y_pred)

submission = pd.DataFrame({
    'id': sample['id'],   # match Kaggle's expected order
    'Tm': y_pred          # predicted target
})

In [ ]:
submission.to_csv('submission1234.csv', index=False)
print("✅ submission.csv created successfully!")